## **Clone Git Repository**

---

Clone git agar dapat load data langsung dari git repository. Dataset yang digunakan didapat kan dari kaggle: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Books.csv

In [1]:
!git clone https://github.com/ziszz/book-recommendation.git

fatal: destination path 'book-recommendation' already exists and is not an empty directory.


## **Import library yang diperlukan**

---



In [2]:
import pandas as pd
import numpy as np
import zipfile
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from zipfile import ZipFile
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## **Data Loading**

---



In [3]:
books = pd.read_csv('/content/book-recommendation/datasets/Books.csv', encoding='utf-8')
ratings = pd.read_csv('/content/book-recommendation/datasets/Ratings.csv', encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [5]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


## **Exploratory Data**

---

### **Variabel-variabel pada dataset adalah sebagai berikut:**

#### **Books.csv**
  * `ISBN`: Kode pengidentifikasian buku yang bersifat unik.
  * `Book-Title`: Judul Buku.
  * `Book-Author`: Nama pengarang buku.
  * `Publisher`: Pihak penerbit buku.
  * `Image-URL-S`: URL yang menautkan ke gambar sampul berukuran small.
  * `Image-URL-M`: URL yang menautkan ke gambar sampul berukuran medium.
  * `Image-URL-L`: URL yang menautkan ke gambar sampul berukuran large.

#### **Ratings.csv**
  * `User-ID`: Nomer unik user yang memberikan rating.
  * `ISBN`: Kode pengidentifikasian buku yang bersifat unik.
  * `Book-Rating`: Skor dari rating yang diberikan.


### **Menghitung jumlah Buku, User dan Rating**

In [6]:
print('Jumlah data buku: ', len(books['ISBN'].unique()))
print('Jumlah data user yang memberikan rating: ', len(ratings['User-ID'].unique()))
print('Jumlah data rating pada buku: ', len(ratings['ISBN'].unique()))

Jumlah data buku:  271360
Jumlah data user yang memberikan rating:  105283
Jumlah data rating pada buku:  340556


### **Mendapatkan info pada data**

Terlihat dari data buku di bawah. Semua kolom data memiliki type data object

In [7]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


Sedangkan, untuk data rating terdapat 2 tipe pada data yaitu numerik (int64) dan object.

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


Pada data terdapat 242135 data unik judul buku. Terlihat bahwa ada beberapa baris dalam data buku memiliki judul buku yang sama. Oleh karena itu, data duplikat perlu dihapus dan hanya memilih baris dengan Judul Buku yang unik.

In [9]:
book_list = books['Book-Title'].value_counts().keys()
jumlah = books['Book-Title'].value_counts()

book_count = pd.DataFrame({'Book-Title': book_list, 'Jumlah': jumlah}).reset_index(drop=True)
book_count

,Book-Title,Jumlah
0,Selected Poems,27
1,Little Women,24
2,Wuthering Heights,21
3,The Secret Garden,20
4,Dracula,20
...,...,...
242130,What Every Kid Should Know,1
242131,The Seventh Enemy (A Brady Coyne Mystery),1
242132,A Brace of Skeet,1
242133,"The Yellow Admiral (O'Brian, Patrick, Aubrey/M...",1


Dari output di bawah, diketahui bahwa nilai maksimum rating adalah 10 dan nilai minimumnya adalah 0. Artinya, skala rating berkisar antara 0 hingga 10.

In [10]:
rating_list = ratings['Book-Rating'].value_counts().keys()
jumlah = ratings['Book-Rating'].value_counts()

rating_count = pd.DataFrame({'Ratings': rating_list, 'Jumlah': jumlah}).reset_index(drop=True)
rating_count

,Ratings,Jumlah
0,0,716109
1,8,103736
2,10,78610
3,7,76457
4,9,67541
5,5,50974
6,6,36924
7,4,8904
8,3,5996
9,2,2759


### **Memeriksa missing value**

Jika dilihat dari data buku dan data rating di bawah. Terdapat sedikit missing value pada data buku, sedangkan pada data rating tidak memiliki missing value.

In [11]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [12]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

## **Data Preparation**

---

### **Menghapus data yang tidak diperlukan**
Sistem rekomendasi ini hanya memerlukan data author dan rating sebagai fitur untuk model. Beberapa kolom data seperti `'Year-Of-Publication', 'Publisher', 'Image-URL-M', 'Image-URL-L'` tidak akan digunakan untuk sistem rekomendasi ini. Jadi data tersebut bisa dihapus.

In [13]:
unused_columns = ['Year-Of-Publication', 'Publisher', 'Image-URL-M', 'Image-URL-L']
books.drop(unused_columns, axis=1, inplace=True)
books

,ISBN,Book-Title,Book-Author,Image-URL-S
0,0195153448,Classical Mythology,Mark P. O. Morford,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,http://images.amazon.com/images/P/0192126040.0...


### **Menggabungkan data buku dan rating**

In [14]:
ratings_with_name = ratings.merge(books,on='ISBN')
new_ratings = ratings_with_name.groupby('Book-Title').sum()['Book-Rating'].reset_index()
new_ratings.rename(columns={'Book-Rating':'Num-Ratings'}, inplace=True)
new_ratings

,Book-Title,Num-Ratings
0,A Light in the Storm: The Civil War Diary of ...,9
1,Always Have Popsicles,0
2,Apple Magic (The Collector's series),0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8
4,Beyond IBM: Leadership Marketing and Finance ...,0
...,...,...
241066,Ã?Â?lpiraten.,0
241067,Ã?Â?rger mit Produkt X. Roman.,21
241068,Ã?Â?sterlich leben.,7
241069,Ã?Â?stlich der Berge.,8


In [53]:
new_books = pd.DataFrame({'Book-Title': books['Book-Title'].unique()})
new_books = pd.merge(new_books, new_ratings, on='Book-Title', how='left')
new_books = new_books.merge(books,on='Book-Title').drop_duplicates('Book-Title', keep=False)
new_books

,Book-Title,Num-Ratings,ISBN,Book-Author,Image-URL-S
2,Clara Callan,69.0,0002005018,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...
3,Decision in Normandy,15.0,0060973129,Carlo D'Este,http://images.amazon.com/images/P/0060973129.0...
4,Flu: The Story of the Great Influenza Pandemic...,47.0,0374157065,Gina Bari Kolata,http://images.amazon.com/images/P/0374157065.0...
5,The Mummies of Urumchi,0.0,0393045218,E. J. W. Barber,http://images.amazon.com/images/P/0393045218.0...
13,PLEADING GUILTY,8.0,0671870432,Scott Turow,http://images.amazon.com/images/P/0671870432.0...
...,...,...,...,...,...
271355,Flashpoints: Promise and Peril in a New World,5.0,0449906736,Robin Wright,http://images.amazon.com/images/P/0449906736.0...
271356,From One to One Hundred,4.0,0525447644,Teri Sloat,http://images.amazon.com/images/P/0525447644.0...
271357,Lily Dale : The True Story of the Town that Ta...,0.0,006008667X,Christine Wicker,http://images.amazon.com/images/P/006008667X.0...
271358,Republic (World's Classics),0.0,0192126040,Plato,http://images.amazon.com/images/P/0192126040.0...


### **Mengatasi missing value**

In [54]:
new_books.isnull().sum()

Book-Title        0
Num-Ratings    1064
ISBN              0
Book-Author       1
Image-URL-S       0
dtype: int64

In [55]:
new_books = new_books.dropna()
new_books.shape

(221163, 5)

In [56]:
new_books.isnull().sum()

Book-Title     0
Num-Ratings    0
ISBN           0
Book-Author    0
Image-URL-S    0
dtype: int64

### **Menyeleksi data**
Data yang akan digunakan yaitu data buku dengan total skor rating dari tiap buku di atas 800. 

In [19]:
final_books = new_books[new_books['Num-Ratings'] > 800]
final_books.drop('ISBN', axis=1, inplace=True)
final_books

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Book-Title,Num-Ratings,Book-Author,Image-URL-S
6,The Kitchen God's Wife,932.0,Amy Tan,http://images.amazon.com/images/P/0399135782.0...
27,The Testament,1962.0,John Grisham,http://images.amazon.com/images/P/0440234743.0...
41,Wild Animus,2551.0,Rich Shapero,http://images.amazon.com/images/P/0971880107.0...
42,Airframe,963.0,Michael Crichton,http://images.amazon.com/images/P/0345402871.0...
47,Timeline,2070.0,MICHAEL CRICHTON,http://images.amazon.com/images/P/0345417623.0...
...,...,...,...,...
12084,Q Is for Quarry,916.0,Sue Grafton,http://images.amazon.com/images/P/0425192725.0...
12287,The Horse Whisperer,1362.0,Nicholas Evans,http://images.amazon.com/images/P/0440222656.0...
12560,Seven Up (A Stephanie Plum Novel),1081.0,Janet Evanovich,http://images.amazon.com/images/P/0312265840.0...
16432,The Talisman,807.0,Stephen King,http://images.amazon.com/images/P/0425105334.0...


## **Model Development**

---

### **Contet-Based Filtering**

In [20]:
data = final_books
data.sample(5)

,Book-Title,Num-Ratings,Book-Author,Image-URL-S
4085,Dolores Claiborne,876.0,Stephen King,http://images.amazon.com/images/P/0451177096.0...
404,Fahrenheit 451,1744.0,Ray Bradbury,http://images.amazon.com/images/P/3257208626.0...
500,Pet Sematary,808.0,Stephen King,http://images.amazon.com/images/P/0451162072.0...
6228,Insomnia,930.0,Stephen King,http://images.amazon.com/images/P/0670855030.0...
4424,Sphere,1026.0,MICHAEL CRICHTON,http://images.amazon.com/images/P/0345353145.0...


#### **Tfid Vectorizer**

In [21]:
tfid = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b\s+\w+")
tfid.fit(data['Book-Author']) 

tfid.get_feature_names() 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aldous huxley',
 'alexander mccall',
 'alice sebold',
 'alice walker',
 'amy tan',
 'andre dubus',
 'anita diamant',
 'anita shreve',
 'ann patchett',
 'anne rice',
 'annie proulx',
 'antoine de',
 'arundhati roy',
 'barbara ehrenreich',
 'barbara kingsolver',
 'benjamin hoff',
 'bernhard schlink',
 'bill bryson',
 'billie letts',
 'caleb carr',
 'charles frazier',
 'chris bohjalian',
 'cornwell patricia',
 'dai sijie',
 'dan brown',
 'dave eggers',
 'dave pelzer',
 'david guterson',
 'david sedaris',
 'dean r',
 'dennis lehane',
 'diana gabaldon',
 'douglas adams',
 'elie wiesel',
 'emma mclaughlin',
 'eoin colfer',
 'eric schlosser',
 'fannie flagg',
 'frank mccourt',
 'george orwell',
 'gregory maguire',
 'gwyn hyman',
 'harlan coben',
 'harper lee',
 'helen fielding',
 'james mcbride',
 'james patterson',
 'jane hamilton',
 'janet evanovich',
 'janet fitch',
 'jeffrey eugenides',
 'jennifer weiner',
 'joanne harris',
 'john berendt',
 'john grisham',
 'john irving',
 'jon krakaue

#### **Transformasi data kedalam bentuk matriks**

In [22]:
tfidf_matrix = tfid.fit_transform(data['Book-Author']) 
tfidf_matrix.shape

(192, 102)

#### **Menghitung Cosine Similarity**

In [24]:
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [28]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['Book-Title'], columns=data['Book-Title'])

### **Mendapatkan rekomendasi buku**
Mendapatkan rekomendasi buku berdasarkan author yang sama dengan buku yang telah dibaca oleh user.

In [51]:
def book_recommendations(book_name, similarity_data=cosine_sim_df, items=data[['Book-Title', 'Book-Author', 'Image-URL-S']], k=5):
  index = similarity_data[book_name].to_numpy().argpartition(range(-1, -(k+1), -1))[::-1]
  closest = similarity_data.columns[index[:k+2]]
  closest = closest.drop(book_name, errors='ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [52]:
book_recommendations('The Testament')

,Book-Title,Book-Author,Image-URL-S
0,The Brethren,JOHN GRISHAM,http://images.amazon.com/images/P/0385497466.0...
1,The Client,John Grisham,http://images.amazon.com/images/P/038542471X.0...
2,The Runaway Jury,JOHN GRISHAM,http://images.amazon.com/images/P/0440221471.0...
3,The Pelican Brief,John Grisham,http://images.amazon.com/images/P/0440214041.0...
4,Skipping Christmas,JOHN GRISHAM,http://images.amazon.com/images/P/0385505833.0...
